In [1]:
# Tipo para almacenar funciones de transferencia y sus derivadas
TransferFunction = NamedTuple{(:f, :f′), Tuple{Function, Function}}

# Funciones de activacion y sus derivadas
σ = TransferFunction((
        # σ
        function (x)
            if x >= 0
                z = exp(-x)
                return 1 / (1 + z)
            else
                z = exp(x)
                return z / (1 + z)
            end
        end,
        
        # σ′
        function (z)
            (1 - z)z
        end
    ))

ReLU = TransferFunction(( 
        (x) -> ifelse(x >= 0, x, 0),
        (z) -> ifelse(z >= 0, 1, 0)
    ))

NamedTuple{(:f, :f′),Tuple{Function,Function}}((getfield(Main, Symbol("##7#9"))(), getfield(Main, Symbol("##8#10"))()))

In [2]:
# Capas de la red
mutable struct Layer{T <: Real}
    size::Tuple{<:Integer, <:Integer}
    tf::TransferFunction
    W::Matrix{T}
    b::Vector{T}
    # Cache output
    z::Vector{T}
    a::Vector{T}
    δ::Vector{T}
    
    Layer{T}(size, tf, W, b, z, a, δ) where {T} = new{T}(size, tf, W, b, z, a, δ)
    function Layer{T}(size::Tuple{<:Integer, <:Integer})  where {T}
        new{T}(size, ReLU,  randn(T, size[1], size[2]) * 1/√size[1], randn(T, size[1]))
    end
    Layer(neurons, inputs) = Layer{Float32}((neurons, inputs))
end

function forward(net::Layer{T}, X::Vector)::Vector{T} where T
    net.z = net.W * X + net.b
    net.a = @. net.tf.f(net.z)

    return net.a
end

forward (generic function with 1 method)

In [3]:
# Probamos la capa de perceptrones
l = Layer(3, 2)
forward(l, Float32[-10., -10.])

3-element Array{Float32,1}:
 1.1703434
 6.0898004
 0.0      

In [4]:
# Funciones de costo
CostFunction = TransferFunction

J = CostFunction((
        (Ŷ, Y) -> (@. (1/2) * (Y - Ŷ) ^ 2), 
        (Ŷ, Y) -> -(Y - Ŷ)
    ))

NamedTuple{(:f, :f′),Tuple{Function,Function}}((getfield(Main, Symbol("##11#13"))(), getfield(Main, Symbol("##12#14"))()))

In [5]:
mutable struct NeuralNetwork{T <: Real}
    layers::Vector{Layer{T}}
    J::CostFunction
    α::T
    error::Vector
    
    NeuralNetwork{T}(layers, J, rate, error) where {T} = new{T}(layers, J, rate, error)
    NeuralNetwork{T}(layers, J, rate) where {T} = new{T}(layers, J, rate)
    NeuralNetwork(layers::Vector{Layer{T}}, J, rate) where {T} = new{T}(layers, J, rate)
end

function forward(net::NeuralNetwork{T}, X::Vector) where T
    a = Vector{Vector{T}}()
    push!(a, convert(Vector{T}, X))
    for layer in net.layers
        push!(a, forward(layer, a[end]))
    end
    return a
end

forward (generic function with 2 methods)

In [6]:
# Calcula la sensibilidad de la capa
function backward(layer::Layer, next::Layer)
    # δ es la sensibilidad de la capa posterior
    z  = layer.z
    f′ = layer.tf.f′
    W  = next.W
    δ  = next.δ
    
    println("W = ", W)
    println("δ = ", δ)
    #pretty_table(f′.(z) .* W' * δ)
    layer.δ = W' .* f′.(z) * δ
    return layer.δ
end

# Calcula el error de toda la red
function backward(net::NeuralNetwork{T}, X::Vector, Y::Vector) where T
    println("backward")
    a         = forward(net, X) # activaciones
    net.error = Vector()

    # Primer capa
    # ∂J/∂W = ∂J/∂a * ∂a/∂z * ∂z/∂W
    Ŷ         = a[end]
    ∂J        = net.J.f′(Ŷ, Y)            # ∂J/∂a = (Y-Ŷ)
    z         = net.layers[end].z         # z = WX + b
    ∂a        = net.layers[end].tf.f′.(z) # ∂a/∂z = f′(z)
    δ         = ∂a .* ∂J                  # multiplicacion elemento a elemento
    ∂z        = a[end-1]                  # ∂z/∂W = activacion de la capa anterior

    net.layers[end].δ = δ

    ∂J∂W = sum(∂z * δ', dims=2)
    println("Capa $(size(net.layers)[1])")

    pushfirst!(net.error, ∂J∂W)           # error de la red

    for l in reverse(eachindex(net.layers[1:end-1]))
        println("Capa $l")
        δ    = backward(net.layers[l], net.layers[l+1])
        ∂J∂W = sum(a[l] * δ', dims=2)

        pushfirst!(net.error, ∂J∂W) # Guardamos el error de la capa
    end

    return ∂J
end

backward (generic function with 2 methods)

In [7]:
# Actualiza los pesos
function update(net::NeuralNetwork)
    println("\nUpdate")
    for (l, layer) in enumerate(net.layers)
        println("Capa $l")
        println("W = ", layer.W)
        println("dJ = ", net.error[l])
        println("b = ", layer.b)
        println("δ = ", layer.δ)
        layer.W .-= net.α * net.error[l]
        layer.b -= net.α * layer.δ
    end
end

update (generic function with 1 method)

In [8]:
net = NeuralNetwork([Layer(2, 2), Layer(2, 2)], J, 0.1)

NeuralNetwork{Float32}(Layer{Float32}[Layer{Float32}((2, 2), NamedTuple{(:f, :f′),Tuple{Function,Function}}((##7#9(), ##8#10())), Float32[-0.633711 0.0338716; 0.614956 0.701119], Float32[-1.49764, -0.977322], #undef, #undef, #undef), Layer{Float32}((2, 2), NamedTuple{(:f, :f′),Tuple{Function,Function}}((##7#9(), ##8#10())), Float32[-0.00324113 -1.18422; 0.271957 -0.112252], Float32[-0.316786, -0.25764], #undef, #undef, #undef)], NamedTuple{(:f, :f′),Tuple{Function,Function}}((getfield(Main, Symbol("##11#13"))(), getfield(Main, Symbol("##12#14"))())), 0.1f0, #undef)

In [9]:
X = [0, 0]
Y = [0, 1]
err = ones(size(Y))
Ŷ = forward(net, X)[end]

while sum(err) > 0.01
    Ŷ = forward(net, X)[end]
    err = net.J.f(Ŷ, [1])
    println("Y, Ŷ: $Y, $Ŷ")
    println("Error: ", sum(err))
    backward(net, X, Y)
    update(net)
end

println("Y, Ŷ: $Y, $Ŷ")
println("Error: ", sum(err))

Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0,

W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.7

W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Y

Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614

W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Y

...

W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Y

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0,

Excessive output truncated after 524295 bytes.

0324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float32[0.0, 0.0]
Error: 1.0
backward
Capa 2
Capa 1
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
δ = Float32[-0.0, -0.0]

Update
Capa 1
W = Float32[-0.633711 0.0338716; 0.614956 0.701119]
dJ = Float32[0.0; 0.0]
b = Float32[-1.49764, -0.977322]
δ = Float32[0.0, 0.0]
Capa 2
W = Float32[-0.00324113 -1.18422; 0.271957 -0.112252]
dJ = Float32[0.0; 0.0]
b = Float32[-0.316786, -0.25764]
δ = Float32[-0.0, -0.0]
Y, Ŷ: [0, 1], Float

InterruptException: InterruptException:

In [ ]:
forward(net, X)

In [ ]:
import PrettyTables: pretty_table

In [ ]:
net.layers[2].W

In [ ]:
methods(NeuralNetwork)

In [ ]:
Array{Layer{Float32},1} == Vector{Layer{Float32}}

In [ ]:
Vector{Vector{Float32}}

In [ ]:
net.α

In [51]:
Σ = sum
Σ([1, 2, 3, 4, 5, 6, 7, 8])

36